# RAG Case Index - 개선된 메타데이터 버전 (v2)

## 개선 내용

| 필드 | 기존 | 개선 |
|------|------|------|
| `src_title` | `"판례"` (모든 청크 동일) | `"판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]"` |
| `title` | 첫 청크만 LLM 요약, 나머지는 `" - 판시사항"` | **모든 청크에 대해 LLM이 실질적인 내용 요약** |

## 목적
- RAG 조회 시 `title`과 `text`를 모두 참조해 정확도를 높임
- LLM context로 `{src_title} - {section}. {text}` 형태로 정확한 원문 제공
- 여러 section을 합쳐 판례 전문 재구성 시 src_title로 출처 명확화

## 1. 환경 설정 및 Import

In [1]:
import os
import re
import glob
import time
import pandas as pd
from tqdm.auto import tqdm
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

# LLM 설정 (요약 생성용) - temperature 0.2로 안정적인 출력
summary_llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.2, num_predict=150)

print("✅ 라이브러리 로드 완료")

c:\Users\Admin\Desktop\Project2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 라이브러리 로드 완료


## 2. 카테고리 키워드 사전

In [2]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 통합 카테고리-키워드 가중치 사전
    """
    return {
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2, '전세보증금': 2,
            '우선매수권': 3, '피해자결정': 2, '경매유예': 3,
            '임대보증금보증': 3, '일부보증': 2, '보증미가입': 3, '동의서': 2,
            '임차권이전': 2, '금융기관담보': 1, '임차권등기명령': 3, 
            '등기촉탁': 2, '기입등기': 2, '전자확정일자': 3,
        },
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1, '2+2': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '직계존비속': 2
        },
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '원상에 회복': 3, '파손': 2, 
            '훼손': 2, '누수': 3, '곰팡이': 2, '보일러': 2, '필요비': 3, 
            '유익비': 3, '비용상환': 2, '보존행위': 2, '청소비': 3, 
            '원상복구': 2, '통상적 마멸': 3, '자연적 마멸': 2, '공제': 3,
            '공용부분': 2, '부속시설': 2, '교체': 1,
        },
        '생활환경_특약': {
            '층간소음': 3, '반려동물': 3, '애완동물': 3, '흡연': 3, '담배': 2,
            '주차': 2, '결로': 2, '방음': 2, '악취': 2, '공동생활': 2, 
            '금지사항': 2, '입주민갈등': 2, '벽지훼손': 2,
            '평온을 해치는 행위': 3, '관리규약': 2
        },
        '관리비_투명성': {
            '관리비': 3, '공공요금': 2, '정액관리비': 3, '세부내역': 2,
            '장기수선충당금': 3, '수선유지비': 2, '관리비인상': 2, '전기료': 1,
            '수도료': 1, '비용 청구': 1, '사용료': 2, '관리주체': 2, '내역공개': 2,
            '수선적립금': 3, '관리단': 2, '분담금': 2
        },
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '연체': 3, '3기': 3,
            '보증금전환': 3, '임차인동의': 2,
        },
        '권리_정보리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '수탁자': 2, '근저당': 3, 
            '저당권': 3, '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, 
            '지방세': 2, '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '확정일자현황': 3, '조세채권': 2, '법정기일': 3, '미납국세열람': 2,
            '사용승인': 2, '무허가': 2, '거짓계약': 2,
            '미납지방세열람': 3, '사해행위취소': 2,
            '본등기': 2, '직권말소': 2, '가등기': 2,
            '정보제공요청': 3, '열람신청': 2, '이해관계인 증명': 2,
        },
        '특약_유효성': {
            '특약': 3, '특약사항': 3, '강행규정': 3, '제10조': 3,
            '효력이 없다': 3, '무효': 3, '불리한 약정': 3, '일방적 포기': 2,
            '편면적 강행규정': 2, '배제': 1, '표준임대차계약서': 3
        },
        '손해배상_책임': {
            '손해배상': 3, '채무불이행': 3, '불법행위': 3, '위약금': 3,
            '과실': 2, '배상액': 2, '지연이자': 2, '이행지체': 2, '위반': 2
        },
        '계약해지_명도': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 3, '이사': 1, '명도': 3, '합의해지': 2, 
            '통지': 1, '3개월': 2, '즉시해지': 3, '내용증명': 2,
            '임대차신고': 3, '거래신고': 3, '신고필증': 2, '등기사항': 2, '정정신청': 2,
        },
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 3, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2, '제소전화해': 3,
            '송달': 2, '강제집행': 3, '배당요구': 3, '매수신청보증': 2,
            '조정신청각하': 2, '출석요구': 1, '서면교부': 1,
            '소액사건': 3, '3,000만원': 3, '이행권고': 2, '공시최고': 1,
            '임차권등기명령신청': 3, '결정송달': 2, '신청서기재사항': 1,
        },
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        }
    }


def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 반환"""
    category_keywords = get_law_category()
    category_scores = {}
    
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword)
            score += count * weight
        if score > 0:
            category_scores[category] = score
    
    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    if not all_categories:
        all_categories = ["기타"]
    
    return all_categories[:top_k] if top_k else all_categories


print(f"✅ 총 {len(get_law_category())}개의 카테고리 정의됨")

✅ 총 13개의 카테고리 정의됨


## 3. 구조적 청킹 함수

In [3]:
def split_case_structurally(text, max_chars=2000):
    """
    판례를 [전문, 주문, 이유] 등 구조적으로 나누되, 
    각 섹션이 max_chars를 넘으면 추가로 분할합니다.
    """
    # 1. 주요 섹션 헤더 찾기
    headers = [
        r'【\s*판\s*시\s*사\s*항\s*】', r'【\s*판\s*결\s*요\s*지\s*】', 
        r'【\s*전\s*문\s*】', r'【\s*주\s*문\s*】', 
        r'【\s*청\s*구\s*취\s*지\s*】', r'【\s*이\s*유\s*】',
        r'【\s*참\s*조\s*조\s*문\s*】', r'【\s*참\s*조\s*판\s*례\s*】'
    ]
    combined_header_regex = '|'.join(headers)
    
    matches = list(re.finditer(combined_header_regex, text))
    
    sections = []
    if not matches:
        sections.append({"section": "본문", "content": text.strip()})
    else:
        # 첫 헤더 이전 내용(제목 등) 처리
        if matches[0].start() > 0:
            sections.append({"section": "서두", "content": text[:matches[0].start()].strip()})
            
        # 각 헤더 사이의 내용 추출
        for i in range(len(matches)):
            start = matches[i].start()
            end = matches[i+1].start() if i+1 < len(matches) else len(text)
            
            header_text = matches[i].group().strip()
            section_name = re.sub(r'[【】\s]', '', header_text)
            content = text[start:end].strip()
            sections.append({"section": section_name, "content": content})

    # 2. 글자 수 제한에 따른 2차 분할
    final_chunks = []
    for sec in sections:
        content = sec['content']
        if len(content) <= max_chars:
            final_chunks.append(sec)
        else:
            num_parts = (len(content) // max_chars) + 1
            for p in range(num_parts):
                part_start = p * max_chars
                part_end = (p + 1) * max_chars
                chunk_part = content[part_start:part_end].strip()
                if chunk_part:
                    final_chunks.append({
                        "section": f"{sec['section']}_p{p+1}",
                        "content": chunk_part
                    })
                    
    return final_chunks


def extract_case_info(filename, content):
    """
    파일명과 본문에서 사건번호를 추출합니다.
    """
    # 사건번호 (파일명에서 추출: 2020나56247)
    case_no_match = re.search(r'\d{4}[가-힣]+\d+', filename)
    case_no = case_no_match.group() if case_no_match else "Unknown_Case"
    
    return case_no

## 4. ⭐ [신규] src_title 추출 함수

In [4]:
def extract_src_title_from_header(header_content, case_no):
    """
    [신규] 서두 section에서 판례의 src_title을 추출합니다.
    
    판례 서두 형식 예시:
    --------------------------------
    임대차보증금
    
    [서울지법 2001. 3. 7. 2000가단148507]
    --------------------------------
    
    출력 형식:
    "판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]"
    
    Parameters:
    - header_content: 서두 section의 텍스트
    - case_no: 사건번호 (예: "2000가단148507")
    
    Returns:
    - src_title: 포맷팅된 출처 제목
    - case_title: 판례 제목 (예: "임대차보증금")
    """
    lines = header_content.strip().split('\n')
    
    # 1. 판례 제목 추출 (첫 번째 비어있지 않은 라인, [ 로 시작하지 않는 것)
    case_title = ""
    for line in lines:
        line = line.strip()
        if line and not line.startswith('[') and not line.startswith('【'):
            # 큰따옴표 제거
            case_title = line.replace('"', '').strip()
            break
    
    # 2. 법원 정보 추출 [서울지법 2001. 3. 7. 2000가단148507]
    court_info = ""
    court_match = re.search(r'\[([^\]]+)\]', header_content)
    if court_match:
        court_info = court_match.group(0)  # [법원 날짜 사건번호] 전체 포함
    
    # 3. 사건번호에서 년도 앞 2자리 제거 (2000가단148507 → 00가단148507)
    short_case_no = case_no
    if len(case_no) >= 4 and case_no[:4].isdigit():
        short_case_no = case_no[2:]  # 년도 앞 2자리 제거
    
    # 4. src_title 조립
    # 형식: "판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]"
    if case_title and court_info:
        src_title = f"판례 {short_case_no} {case_title} {court_info}"
    elif case_title:
        src_title = f"판례 {short_case_no} {case_title}"
    elif court_info:
        src_title = f"판례 {short_case_no} {court_info}"
    else:
        src_title = f"판례 {short_case_no}"
    
    return src_title, case_title

In [5]:
# 테스트: src_title 추출
test_headers = [
    ('''임대차보증금

[서울지법 2001. 3. 7. 2000가단148507]''', "2000가단148507"),
    
    ('''부당이득금·보증금반환

[대법원 2023. 9. 27. 2022다246610, 246627]''', "2022다246610"),
    
    ('''임대차보증금반환[주택 임차인의 점유 상실 후 마쳐진 임차권등기의 대항력이 문제된 사건]

[대법원 2025. 4. 15. 2024다326398]''', "2024다326398"),
]

print("=" * 70)
print("[src_title 추출 테스트]")
print("=" * 70)
for header, case_no in test_headers:
    src_title, case_title = extract_src_title_from_header(header, case_no)
    print(f"\n사건번호: {case_no}")
    print(f"판례제목: {case_title}")
    print(f"src_title: {src_title}")

[src_title 추출 테스트]

사건번호: 2000가단148507
판례제목: 임대차보증금
src_title: 판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]

사건번호: 2022다246610
판례제목: 부당이득금·보증금반환
src_title: 판례 22다246610 부당이득금·보증금반환 [대법원 2023. 9. 27. 2022다246610, 246627]

사건번호: 2024다326398
판례제목: 임대차보증금반환[주택 임차인의 점유 상실 후 마쳐진 임차권등기의 대항력이 문제된 사건]
src_title: 판례 24다326398 임대차보증금반환[주택 임차인의 점유 상실 후 마쳐진 임차권등기의 대항력이 문제된 사건] [주택 임차인의 점유 상실 후 마쳐진 임차권등기의 대항력이 문제된 사건]


## 5. ⭐ [개선] 섹션별 LLM 요약 함수

In [6]:
def summarize_section_with_llm(content, section_name, llm=None):
    """
    [개선] 모든 section에 대해 실질적인 내용 요약을 생성합니다.
    
    기존 문제점: 첫 번째 청크만 LLM 요약, 나머지는 "- 판시사항" 형태
    개선: 모든 청크에 대해 LLM이 실질적인 내용 요약 생성
    
    Parameters:
    - content: 청크 텍스트
    - section_name: 섹션 이름 (서두, 판시사항, 판결요지, 이유 등)
    - llm: ChatOllama 인스턴스 (None이면 새로 생성)
    
    Returns:
    - 요약된 제목 문자열
    """
    if llm is None:
        llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.2, num_predict=150)
    
    # 내용이 너무 짧으면 그대로 반환
    if len(content) < 50:
        return content[:100]
    
    # 섹션별 맞춤 프롬프트
    section_prompts = {
        "서두": "다음 판례의 핵심 쟁점을 1문장으로 요약하세요.",
        "판시사항": "다음 판시사항의 핵심 법리를 1문장으로 요약하세요.",
        "판결요지": "다음 판결요지의 핵심 결론을 1문장으로 요약하세요.",
        "이유": "다음 판결 이유의 핵심 논거를 1문장으로 요약하세요.",
        "전문": "다음 전문의 핵심 내용을 1문장으로 요약하세요.",
        "주문": "다음 판결 주문의 핵심 결정을 1문장으로 요약하세요.",
        "청구취지": "다음 청구취지의 핵심 요구사항을 1문장으로 요약하세요.",
        "참조조문": "참조된 주요 조문명을 나열하세요.",
        "참조판례": "참조된 주요 판례의 쟁점을 1문장으로 요약하세요.",
    }
    
    # 섹션 이름에서 _p1, _p2 등 제거
    base_section = re.sub(r'_p\d+$', '', section_name)
    
    # 기본 프롬프트 또는 섹션별 맞춤 프롬프트
    instruction = section_prompts.get(base_section, 
        "다음 판례 내용의 핵심을 1문장으로 요약하세요.")
    
    prompt = f"""당신은 대한민국 법률 전문가입니다.
{instruction}

[규칙]
1. 50자 이내로 간결하게 요약
2. 핵심 법리/결론/쟁점만 포함
3. 평서문 형태로 작성
4. 요약문만 출력 (설명 금지)

[내용]:
{content[:1500]}

[요약]:"""

    try:
        ai_message = llm.invoke(prompt)
        summary = ai_message.content.strip()
        
        # 후처리: 불필요한 접두사 제거
        prefixes_to_remove = [
            "요약:", "요약 :", "[요약]:", "[요약] :",
            "답변:", "답변 :", "결과:", "결과 :",
            "- ", "• ", "* ",
        ]
        for prefix in prefixes_to_remove:
            if summary.lower().startswith(prefix.lower()):
                summary = summary[len(prefix):].strip()
        
        # 너무 길면 자르기
        if len(summary) > 100:
            summary = summary[:97] + "..."
        
        return summary if summary else section_name
        
    except Exception as e:
        print(f"   ⚠️ LLM 요약 실패 ({section_name}): {str(e)[:30]}")
        return section_name

## 6. ⭐ [개선] 단일 판례 파일 처리 함수

In [7]:
def process_single_case_file_v2(file_path, priority=9, use_llm_summary=True, llm=None):
    """
    [개선된 버전] 판례 파일 하나를 처리하여 Documents 리스트를 반환합니다.
    
    메타데이터 변경 사항:
    - src_title: "판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]"
    - title: 모든 section에 대해 LLM이 실질적인 내용 요약 생성
    
    Parameters:
    - file_path: docx 파일 경로
    - priority: RAG 참조 우선순위 (기본 9)
    - use_llm_summary: LLM 요약 사용 여부
    - llm: 재사용할 LLM 인스턴스 (None이면 새로 생성)
    """
    file_name = os.path.basename(file_path)
    
    # 1. 로드
    loader = Docx2txtLoader(file_path)
    docs = loader.load()
    full_text = docs[0].page_content
    
    # 2. 사건번호 추출
    case_no = extract_case_info(file_name, full_text)
    
    # 3. 구조적 청킹
    structural_chunks = split_case_structurally(full_text, max_chars=2000)
    
    # 4. [신규] 서두에서 src_title 추출
    src_title = f"판례 {case_no}"  # 기본값
    case_title = ""
    
    for chunk_data in structural_chunks:
        if chunk_data['section'] == '서두':
            src_title, case_title = extract_src_title_from_header(
                chunk_data['content'], case_no
            )
            break
    
    # 5. LLM 인스턴스 준비 (재사용)
    if use_llm_summary and llm is None:
        try:
            llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.2, num_predict=150)
        except Exception as e:
            print(f"   ⚠️ LLM 초기화 실패: {e}")
            use_llm_summary = False
    
    # 6. 각 청크 처리
    processed_docs = []
    for i, chunk_data in enumerate(structural_chunks):
        section = chunk_data['section']
        content = chunk_data['content']
        
        # (1) 카테고리 분류
        categories = categorize_content(content)
        if not categories:
            categories = ["기타"]
        
        # (2) [개선] 모든 section에 대해 LLM 요약
        if use_llm_summary:
            ai_title = summarize_section_with_llm(content, section, llm)
        else:
            # LLM 미사용 시 본문 앞 100자로 대체
            clean_content = re.sub(r'【[^】]*】', '', content).strip()
            ai_title = clean_content[:100] + "..." if len(clean_content) > 100 else clean_content

        # (3) 메타데이터 조립
        metadata = {
            "chunk_id": f"PREC_{case_no}_{i:03d}",
            "priority": int(priority),
            "category": categories,
            "case_no": case_no,
            "title": str(ai_title).strip(),      # [개선] 모든 section에 LLM 요약
            "section": section,
            "src_title": str(src_title).strip(), # [개선] 상세 출처 정보
            "source": file_name
        }
        
        doc = Document(page_content=content, metadata=metadata)
        processed_docs.append(doc)
        
    print(f"✅ 처리 완료: {case_no} ({len(processed_docs)} chunks)")
    print(f"   src_title: {src_title[:60]}...")
    return processed_docs

## 7. 다중 파일 일괄 처리

In [8]:
def process_multiple_case_files_v2(case_dir, output_dir='./data/1-2_chunked/', use_llm_summary=True):
    """
    지정된 디렉토리의 모든 판례 파일을 처리합니다.
    
    Parameters:
    - case_dir: 판례 docx 파일들이 있는 디렉토리
    - output_dir: CSV 저장 디렉토리
    - use_llm_summary: LLM 요약 사용 여부
    
    Returns:
    - all_docs: 모든 처리된 Document 리스트
    """
    # 판례 파일 목록 가져오기
    case_files = glob.glob(os.path.join(case_dir, "판례_*.docx"))
    
    if not case_files:
        print(f"❌ '{case_dir}'에서 판례 파일을 찾을 수 없습니다.")
        return []
    
    print(f"🚀 총 {len(case_files)}개의 판례 파일을 처리합니다...")
    
    # LLM 인스턴스 미리 생성 (재사용)
    llm = None
    if use_llm_summary:
        try:
            llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.2, num_predict=150)
            print("   LLM 인스턴스 생성 완료")
        except Exception as e:
            print(f"   ⚠️ LLM 초기화 실패: {e}")
            use_llm_summary = False
    
    all_docs = []
    
    for file_path in tqdm(case_files, desc="판례 처리 중"):
        try:
            docs = process_single_case_file_v2(
                file_path=file_path,
                priority=9,
                use_llm_summary=use_llm_summary,
                llm=llm
            )
            all_docs.extend(docs)
        except Exception as e:
            print(f"❌ 오류 발생 ({os.path.basename(file_path)}): {e}")
            continue
    
    # CSV 저장
    if all_docs and output_dir:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        df_data = []
        for doc in all_docs:
            row = doc.metadata.copy()
            row['page_content'] = doc.page_content
            row['category'] = str(row['category'])
            df_data.append(row)
        
        df = pd.DataFrame(df_data)
        timestamp = time.strftime("%Y%m%d_%H%M")
        csv_path = os.path.join(output_dir, f"processed_case_data_v2_{timestamp}.csv")
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ CSV 저장 완료: {csv_path}")
    
    print(f"\n✨ 모든 작업 완료! 총 {len(all_docs)}개의 청크가 처리되었습니다.")
    return all_docs

In [14]:
# ==========================================
# 실행: 판례 파일 처리
# ==========================================

# 판례 디렉토리 경로 설정 (실제 경로로 수정하세요)
CASE_DIR = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\case\판례\docx'

# 일괄 처리 실행
all_case_docs = process_multiple_case_files_v2(
    case_dir=CASE_DIR,
    output_dir='./data/1-2_chunked/',
    use_llm_summary=True
)

🚀 총 85개의 판례 파일을 처리합니다...
   LLM 인스턴스 생성 완료


판례 처리 중:   1%|          | 1/85 [01:37<2:16:25, 97.45s/it]

✅ 처리 완료: 2000가단148507 (10 chunks)
   src_title: 판례 00가단148507 임대차보증금 (판례, 2000가단148507) [서울지법 2001. 3. 7. 20...


판례 처리 중:   2%|▏         | 2/85 [04:02<2:53:37, 125.51s/it]

✅ 처리 완료: 2000다24184 (9 chunks)
   src_title: 판례 00다24184 임대차보증금·건물명도 (판례, 2000다24184, 24191) [대법원 2002. 3...


...

✅ 처리 완료: 2024다221455 (8 chunks)
   src_title: 판례 24다221455 건물인도[주택임대차보호법에 따른 임차... (판례, 2024다221455) [주택임대...


판례 처리 중:  99%|█████████▉| 84/85 [3:58:23<02:56, 176.15s/it]

✅ 처리 완료: 2024다305087 (11 chunks)
   src_title: 판례 24다305087 공제금등청구의소 (판례, 2024다305087) [대법원 2025. 12. 4. 20...


판례 처리 중: 100%|██████████| 85/85 [4:01:21<00:00, 170.37s/it]

✅ 처리 완료: 2024다326398 (10 chunks)
   src_title: 판례 24다326398 임대차보증금반환[주택 임차인의 점유 ... (판례, 2024다326398) [주택 임...

✅ CSV 저장 완료: ./data/1-2_chunked/processed_case_data_v2_20260127_2043.csv

✨ 모든 작업 완료! 총 796개의 청크가 처리되었습니다.


## 8. Pinecone Upsert

In [15]:
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

# API 키 로드
pc_api_key = os.getenv("PINECONE_API_KEY", "").strip()
up_api_key = os.getenv("UPSTAGE_API_KEY", "").strip()

if not pc_api_key or not up_api_key:
    print("❌ [오류] API 키를 로드할 수 없습니다. .env 파일을 확인하세요.")
else:
    print(f"🔑 Pinecone API Key: {pc_api_key[:8]}***")
    print(f"🔑 Upstage API Key: {up_api_key[:8]}***")

🔑 Pinecone API Key: pcsk_3kd***
🔑 Upstage API Key: up_ABxVS***


In [17]:
def upsert_cases_to_pinecone(all_docs, index_name="case-index-final", pc_api_key=None, up_api_key=None):
    """
    처리된 판례 Documents를 Pinecone에 업로드합니다.
    """
    print(f"\n🔄 Pinecone '{index_name}' 인덱스에 업로드를 시작합니다...")
    print(f"📦 총 {len(all_docs)}개의 판례 청크가 대기 중입니다.")
    
    # Pinecone 클라이언트 초기화
    pc = Pinecone(api_key=pc_api_key)
    
    # 임베딩 모델 설정 (Upstage Solar: 4096 차원)
    embedding = UpstageEmbeddings(
        model="solar-embedding-1-large-passage",
        api_key=up_api_key
    )
    
    # 인덱스 존재 여부 확인 및 생성
    if index_name not in pc.list_indexes().names():
        print(f"⚙️ '{index_name}' 인덱스가 없어 새로 생성합니다 (차원: 4096)...")
        pc.create_index(
            name=index_name,
            dimension=4096,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        while not pc.describe_index(index_name).status['ready']:
            time.sleep(1)
        print("   - 인덱스 생성 및 활성화 완료")
    else:
        print(f"ℹ️ '{index_name}' 인덱스가 이미 존재합니다.")
    
    # 업로드
    try:
        database = PineconeVectorStore.from_documents(
            documents=all_docs,
            embedding=embedding,
            index_name=index_name,
            pinecone_api_key=pc_api_key
        )
        print(f"\n🎉 [성공] 모든 판례 청크가 '{index_name}'에 저장되었습니다!")
        return database
        
    except Exception as e:
        print(f"\n🔥 업로드 중 에러 발생: {e}")
        return None

In [18]:
# ==========================================
# 실행: case-index-final에 업로드
# ==========================================

case_db = upsert_cases_to_pinecone(
    all_docs=all_case_docs,
    index_name="case-index",
    pc_api_key=pc_api_key,
    up_api_key=up_api_key
)


🔄 Pinecone 'case-index' 인덱스에 업로드를 시작합니다...
📦 총 796개의 판례 청크가 대기 중입니다.
⚙️ 'case-index' 인덱스가 없어 새로 생성합니다 (차원: 4096)...
   - 인덱스 생성 및 활성화 완료

🎉 [성공] 모든 판례 청크가 'case-index'에 저장되었습니다!


## 9. ⭐ [신규] 판례 전문 재구성 및 LLM Context 포맷팅

In [19]:
def reconstruct_case_full_text(docs_by_case_no):
    """
    같은 case_no를 가진 여러 section 청크들을 합쳐 판례 전문을 재구성합니다.
    
    Parameters:
    - docs_by_case_no: {case_no: [doc1, doc2, ...]} 형태의 딕셔너리
    
    Returns:
    - 재구성된 전문 딕셔너리 {case_no: full_text}
    """
    # section 순서 정의
    section_order = ['서두', '판시사항', '판결요지', '참조조문', '참조판례', '전문', '주문', '청구취지', '이유']
    
    full_texts = {}
    
    for case_no, docs in docs_by_case_no.items():
        # section 기준으로 정렬
        def get_section_order(doc):
            section = doc.metadata.get('section', '')
            base_section = re.sub(r'_p\d+$', '', section)
            try:
                return section_order.index(base_section)
            except ValueError:
                return len(section_order)
        
        sorted_docs = sorted(docs, key=get_section_order)
        
        # src_title 가져오기
        src_title = sorted_docs[0].metadata.get('src_title', f'판례 {case_no}')
        
        # 전문 재구성
        text_parts = [f"=== {src_title} ==="]
        for doc in sorted_docs:
            section = doc.metadata.get('section', '')
            text_parts.append(f"\n[{section}]\n{doc.page_content}")
        
        full_texts[case_no] = "\n".join(text_parts)
    
    return full_texts


def format_case_for_llm_context(docs):
    """
    검색된 판례 청크들을 LLM context용 문자열로 포맷팅합니다.
    같은 case_no의 청크들은 합쳐서 표시합니다.
    
    형식: "{src_title}\n{section}: {text}"
    """
    # case_no별로 그룹화
    docs_by_case = {}
    for doc in docs:
        case_no = doc.metadata.get('case_no', 'unknown')
        if case_no not in docs_by_case:
            docs_by_case[case_no] = []
        docs_by_case[case_no].append(doc)
    
    # 각 판례별로 포맷팅
    formatted_texts = []
    for i, (case_no, case_docs) in enumerate(docs_by_case.items()):
        src_title = case_docs[0].metadata.get('src_title', f'판례 {case_no}')
        
        # 각 section 내용 결합
        section_texts = []
        for doc in case_docs:
            section = doc.metadata.get('section', '')
            content_preview = doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content
            section_texts.append(f"[{section}] {content_preview}")
        
        context_text = f"[{i+1}] {src_title}\n" + "\n".join(section_texts)
        formatted_texts.append(context_text)
    
    return "\n\n" + "="*60 + "\n\n".join(formatted_texts)


def format_case_for_display(docs):
    """
    검색된 판례를 사용자 화면 표시용으로 포맷팅합니다.
    
    Returns:
    - 딕셔너리 리스트 [{rank, src_title, section, title, text_preview, ...}, ...]
    """
    results = []
    for i, doc in enumerate(docs):
        results.append({
            "rank": i + 1,
            "src_title": doc.metadata.get('src_title', ''),
            "case_no": doc.metadata.get('case_no', ''),
            "section": doc.metadata.get('section', ''),
            "title": doc.metadata.get('title', ''),
            "text_preview": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
            "priority": doc.metadata.get('priority', 9),
            "category": doc.metadata.get('category', [])
        })
    return results

## 10. 테스트 및 결과 확인

In [21]:
# ==========================================
# 단일 파일 테스트
# ==========================================

# 테스트할 판례 파일 경로
TEST_FILE = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\case\판례\docx\판례_2000가단148507.docx'

# 테스트 실행
test_docs = process_single_case_file_v2(
    file_path=TEST_FILE,
    priority=9,
    use_llm_summary=True
)

# 결과 확인
print("\n" + "=" * 70)
print("[개선된 메타데이터 확인]")
print("=" * 70)
for doc in test_docs[:3]:
    print(f"\n--- {doc.metadata['section']} ---")
    print(f"src_title: {doc.metadata['src_title']}")
    print(f"title: {doc.metadata['title']}")
    print(f"category: {doc.metadata['category']}")

✅ 처리 완료: 2000가단148507 (10 chunks)
   src_title: 판례 00가단148507 임대차보증금 (판례, 2000가단148507) [서울지법 2001. 3. 7. 20...

[개선된 메타데이터 확인]

--- 서두 ---
src_title: 판례 00가단148507 임대차보증금 (판례, 2000가단148507) [서울지법 2001. 3. 7. 2000가단148507]
title: 임대차보증금 반환 여부는 임차인의 계약 해지 및 임대인의 반환 의무를 중심으로 판단되었으며, 계약 조건과 해지 시점의 적법성에 따라 결정되었다.
category: ['보증금_대항력']

--- 판시사항 ---
src_title: 판례 00가단148507 임대차보증금 (판례, 2000가단148507) [서울지법 2001. 3. 7. 2000가단148507]
title: 임차인의 이의 제기에도 불구하고, 주택임대차보호법상 임대인의 지위승계는 구속력이 있어 임차인은 이를 거부할 수 없다.
category: ['임차권_승계']

--- 판결요지 ---
src_title: 판례 00가단148507 임대차보증금 (판례, 2000가단148507) [서울지법 2001. 3. 7. 2000가단148507]
title: 임대차보호법에 따라 임차주택의 양수인은 임대인 지위를 승계하며, 임차인은 임대인 지위 승계에 대한 이의 없이 보증금 반환 의무를 지게 된다.
category: ['임차권_승계', '보증금_대항력', '분쟁해결']


In [22]:
# ==========================================
# 기존 vs 개선 비교
# ==========================================

print("""
┌─────────────────────────────────────────────────────────────────────────┐
│                     기존 vs 개선 메타데이터 비교                          │
├─────────────────────────────────────────────────────────────────────────┤
│ 필드       │ 기존                          │ 개선                        │
├─────────────────────────────────────────────────────────────────────────┤
│ src_title  │ "판례"                        │ "판례 00가단148507           │
│            │                               │  임대차보증금               │
│            │                               │  [서울지법 2001.3.7         │
│            │                               │   2000가단148507]"          │
├─────────────────────────────────────────────────────────────────────────┤
│ title      │ 첫 청크: LLM 요약             │ 모든 청크에 LLM 요약        │
│ (서두)     │ "임대차보증금 관련 판결 요약"  │ "임차주택 양수인에게         │
│            │                               │  임대인 지위가 승계된다"     │
├─────────────────────────────────────────────────────────────────────────┤
│ title      │ " - 판시사항"                 │ "임대인 지위승계에 대한      │
│ (판시사항) │                               │  임차인의 이의 불가"         │
├─────────────────────────────────────────────────────────────────────────┤
│ title      │ " - 판결요지"                 │ "주택임대차보호법상          │
│ (판결요지) │                               │  양수인은 임대인 지위 승계"  │
├─────────────────────────────────────────────────────────────────────────┤
│ title      │ " - 이유_p1"                  │ "원고의 보증금반환청구를     │
│ (이유)     │                               │  기각한 판단 근거"          │
└─────────────────────────────────────────────────────────────────────────┘
""")


┌─────────────────────────────────────────────────────────────────────────┐
│                     기존 vs 개선 메타데이터 비교                          │
├─────────────────────────────────────────────────────────────────────────┤
│ 필드       │ 기존                          │ 개선                        │
├─────────────────────────────────────────────────────────────────────────┤
│ src_title  │ "판례"                        │ "판례 00가단148507           │
│            │                               │  임대차보증금               │
│            │                               │  [서울지법 2001.3.7         │
│            │                               │   2000가단148507]"          │
├─────────────────────────────────────────────────────────────────────────┤
│ title      │ 첫 청크: LLM 요약             │ 모든 청크에 LLM 요약        │
│ (서두)     │ "임대차보증금 관련 판결 요약"  │ "임차주택 양수인에게         │
│            │                               │  임대인 지위가 승계된다"     │
├─────────────────────────────────────────────────────────────────────────┤
│

## 11. 활용 예시

In [23]:
# ==========================================
# LLM Context 포맷 예시
# ==========================================

print("""
[LLM Context 형식 예시]

============================================================
[1] 판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]

[판시사항] 주택임대차보호법상의 임차주택에 대한 임대인의 지위승계에 
대하여 임차인이 이의를 함으로써 승계되는 임대차관계의 구속을 면할 수 
있는지 여부(소극)...

[판결요지] 주거용건물의 임대차관계에서는 주택임대차보호법 제3조 제1항, 
제2항에 의하여 임차주택의 양수인이 당연히 임대인의 지위를 승계한 것으로 
보고, 양도인은 임차보증금반환채무를 면한다고 보아야 할 것이지...

============================================================
[2] 판례 22다246610 부당이득금·보증금반환 [대법원 2023. 9. 27. 2022다246610]

[판시사항] 임차권등기가 마쳐진 주택을 임차한 임차인에게도 소액임차보증금에 
관한 최우선변제권을 제외한 대항력과 우선변제권을 인정할 수 있는지 여부(적극)
...
""")


[LLM Context 형식 예시]

[1] 판례 00가단148507 임대차보증금 [서울지법 2001. 3. 7. 2000가단148507]

[판시사항] 주택임대차보호법상의 임차주택에 대한 임대인의 지위승계에 
대하여 임차인이 이의를 함으로써 승계되는 임대차관계의 구속을 면할 수 
있는지 여부(소극)...

[판결요지] 주거용건물의 임대차관계에서는 주택임대차보호법 제3조 제1항, 
제2항에 의하여 임차주택의 양수인이 당연히 임대인의 지위를 승계한 것으로 
보고, 양도인은 임차보증금반환채무를 면한다고 보아야 할 것이지...

[2] 판례 22다246610 부당이득금·보증금반환 [대법원 2023. 9. 27. 2022다246610]

[판시사항] 임차권등기가 마쳐진 주택을 임차한 임차인에게도 소액임차보증금에 
관한 최우선변제권을 제외한 대항력과 우선변제권을 인정할 수 있는지 여부(적극)
...

